In [1]:
#!pip install tensorflow
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, LSTM,BatchNormalization,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2
import gc
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error,mean_squared_error

# Lstm model after using encoder

In [41]:
input_step=10 #to be changed after to 10,10
output_step=10
fsize=64

In [42]:
X1=np.load("X11_"+str(input_step)+".npy")
X2=np.load("X22_"+str(input_step)+".npy")


In [43]:
np.shape(X1)

(56936, 640)

In [44]:
Y1=np.load("Y1_"+str(output_step)+".npy",allow_pickle=True)
Y2=np.load("Y2_"+str(output_step)+".npy",allow_pickle=True)

In [45]:
X1=np.float32(X1)
X2=np.float32(X2)
Y1=np.float32(Y1)
Y2=np.float32(Y2)

In [46]:
X1=np.reshape(X1,(X1.shape[0],input_step,fsize))
X2=np.reshape(X2,(X2.shape[0],input_step,fsize))

In [47]:
X1.shape

(56936, 10, 64)

In [48]:
XT=np.concatenate((X1, X2), axis=0)

In [49]:
YT=np.concatenate((Y1, Y2), axis=0)

In [50]:
def createModel(inputShape,output,init):
# define model
    model = Sequential()
    model.add(LSTM(100, activation='relu',input_shape=inputShape,return_sequences=True, kernel_initializer=init))
    model.add(LSTM(100,activation='relu', return_sequences=False, kernel_initializer=init))
#     model.add(LSTM(64,activation='relu', return_sequences=False))
#     model.add(Dense(16, activation='linear'))
#     model.add(Dropout(0.3))
    model.add(Dense(output,activation="sigmoid", kernel_initializer=init))

    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,loss='mae',metrics=['mae','mse',tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.MeanAbsolutePercentageError()])
    # model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mse'])
    return model

In [51]:
init=[tf.keras.initializers.GlorotUniform(),tf.keras.initializers.Zeros(),tf.keras.initializers.RandomNormal,tf.keras.initializers.RandomUniform,tf.keras.initializers.HeNormal]


In [52]:
time_steps = input_step  # Number of time steps
feature_length = 64  # Length of each feature vector
input_shape = (time_steps, feature_length)

model=createModel(input_shape,output_step,init[0])

C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [53]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_24 (LSTM)              (None, 10, 100)           66000     
                                                                 
 lstm_25 (LSTM)              (None, 100)               80400     
                                                                 
 dense_12 (Dense)            (None, 10)                1010      
                                                                 
Total params: 147,410
Trainable params: 147,410
Non-trainable params: 0
_________________________________________________________________


In [36]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [19]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error,mean_squared_error
# load your data
X = XT  # your features
y = YT  # your target variable

# set up the k-fold cross-validation object
k =5   # number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# initialize a list to store the validation accuracies
for i in range(0,len(init)):
    val_mae = []
    val_mse = []
    val_mape=[]

    # loop over the k folds
    for train_index, val_index in kf.split(X):
        # split the data into training and validation sets
        X_train, y_train = X[train_index], y[train_index]
        X_val, y_val = X[val_index], y[val_index]
        model=createModel(input_shape,output_step,init[i]) #each time we create a model the weights will be initailize with default value given by keras



        # train your model on the training set
        model.fit(X_train, y_train)
        #model.save("lstm_model_withencoder_"+str(input_step)+"_F"+str(i)+".h5")

        # evaluate the model on the validation set and store the mse
        y_pred = model.predict(X_val)
        val_acc = mean_absolute_error(y_val, y_pred)
        val_mae.append(val_acc)
        val_acc = mean_squared_error(y_val, y_pred)
        val_mse.append(val_acc)
        val_acc = MAPE(y_val, y_pred)
        val_mape.append(val_acc)
        del X_train,y_train,X_val,y_val
        gc.collect()

    # print the mean and standard deviation of the validation mse
    print("initailizer number",i)
    print(f'Validation mae: {np.mean(val_mae):.5f} ± {np.std(val_mae):.5f}')
    print(f'Validation mse: {np.mean(val_mse):.5f} ± {np.std(val_mse):.5f}')
    print(f'Validation mape: {np.mean(val_mape):.5f} ± {np.std(val_mape):.5f}')

C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 2s 2ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 2s 2ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 2s 2ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 2s 3ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 2s 2ms/step
initailizer number 0
Validation mae: 0.00789 ± 0.00020
Validation mse: 0.00015 ± 0.00001
Validation mape: 0.84111 ± 0.02025
700/700 [==============================] - 2s 2ms/step
initailizer number 1
Validation mae: 0.39601 ± 0.00024
Validation mse: 0.15877 ± 0.00016
Validation mape: 40.88889 ± 0.01635
700/700 [==============================] - 2s 3ms/step
initailizer number 2
Validation mae: 0.01928 ± 0.00848
Validation mse: 0.00122 ± 0.00099
Validation mape: 2.09469 ± 0.94738
700/700 [==============================] - 2s 3ms/step
initailizer number 3
Validation mae: 0.03100 ± 0.00906
Validation mse: 0.00259 ± 0.00110
Validation mape: 3.40864 ± 1.00981
700/700 [==============================] - 2s 3ms/step
initailizer number 4
Validation mae: 0.00797 ± 0.00134
Validation mse: 0.00016 ± 0.00005
Validation mape: 0.85243 ± 0.14714


In [54]:
X = XT  # your features
y = YT  # your target variable

# set up the k-fold cross-validation object
k =5   # number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)
val_mae = []
val_mse = []
val_mape=[]
i=0

    # loop over the k folds
for train_index, val_index in kf.split(X):
        # split the data into training and validation sets
        X_train, y_train = X[train_index], y[train_index]
        X_val, y_val = X[val_index], y[val_index]
        model=createModel(input_shape,output_step,init[0]) #each time we create a model the weights will be initailize with default value given by keras



        # train your model on the training set
        model.fit(X_train, y_train)
        model.save("lstm_model_withencoder_"+str(input_step)+"_F"+str(i)+".h5")

        # evaluate the model on the validation set and store the mse
        y_pred = model.predict(X_val)
        val_acc = mean_absolute_error(y_val, y_pred)
        val_mae.append(val_acc)
        val_acc = mean_squared_error(y_val, y_pred)
        val_mse.append(val_acc)
        val_acc = MAPE(y_val, y_pred)
        val_mape.append(val_acc)
        del X_train,y_train,X_val,y_val
        gc.collect()
        i=i+1

    # print the mean and standard deviation of the validation mse
    
print(f'Validation mae: {np.mean(val_mae):.5f} ± {np.std(val_mae):.5f}')
print(f'Validation mse: {np.mean(val_mse):.5f} ± {np.std(val_mse):.5f}')
print(f'Validation mape: {np.mean(val_mape):.5f} ± {np.std(val_mape):.5f}')

700/700 [==============================] - 2s 2ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 2s 2ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 2s 3ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 2s 3ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 2s 3ms/step
Validation mae: 0.00957 ± 0.00110
Validation mse: 0.00022 ± 0.00005
Validation mape: 1.02451 ± 0.12463


In [55]:
val_mae,val_mape

([0.008731315, 0.008841053, 0.011280319, 0.010471091, 0.008544832],
 [0.9324093349277973,
  0.9392650797963142,
  1.2160448357462883,
  1.1295100674033165,
  0.9053208865225315])

# CNN-LSTM model

In [67]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Flatten, Reshape, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
import tensorflow as tf

def create_model2(init):
    
# Define CNN-LSTM model architecture
    model2 = Sequential()

    # Convolutional layers
    model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(input_step, 6, 500), kernel_initializer=init))
    model2.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten and reshape for LSTM
    model2.add(Flatten())
    model2.add(Reshape((8, 32))) #8,32 for 10 input window

    # LSTM layers
    model2.add(LSTM(32, return_sequences=True, kernel_initializer=init))
    model2.add(LSTM(32, return_sequences=False, kernel_initializer=init))


    # Dense layer
    model2.add(Dense(output_step, activation='sigmoid', kernel_initializer=init))

    # Compile model
    opt = Adam(learning_rate=0.001)
    model2.compile(loss='mean_absolute_error', optimizer=opt, metrics=['mae', 'mse', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsolutePercentageError()])

    
    return model2



In [57]:
X11=np.load("X1_"+str(input_step)+".npy")
X22=np.load("X2_"+str(input_step)+".npy")

In [58]:
Y1=np.load("Y1_"+str(output_step)+".npy",allow_pickle=True)
Y2=np.load("Y2_"+str(output_step)+".npy",allow_pickle=True)

In [7]:
X11.shape,X22.shape

((52316, 25, 6, 500), (51922, 25, 6, 500))

In [59]:
X11=np.float32(X11)
X22=np.float32(X22)

In [60]:
Y1=np.float32(Y1)
Y2=np.float32(Y2)

In [61]:
Y1.shape

(56936, 10)

In [62]:
XT=np.concatenate((X11, X22), axis=0)

In [63]:
XT.shape

(111938, 10, 6, 500)

In [64]:
YT=np.concatenate((Y1, Y2), axis=0)

In [65]:
YT.shape

(111938, 10)

In [34]:
X = XT  # your features
y = YT  # your target variable

# set up the k-fold cross-validation object
k =5   # number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# initialize a list to store the validation accuracies
for i in range(0,len(init)):
    val_mae = []
    val_mse = []
    val_mape=[]

    # loop over the k folds
    for train_index, val_index in kf.split(X):
        # split the data into training and validation sets
        X_train, y_train = X[train_index], y[train_index]
        X_val, y_val = X[val_index], y[val_index]
        model2=create_model2(init[i]) #each time we create a model the weights will be initailize with default value given by keras



        # train your model on the training set
        model2.fit(X_train, y_train)
        #model.save("lstm_model_withencoder_"+str(input_step)+"_F"+str(i)+".h5")

        # evaluate the model on the validation set and store the mse
        y_pred = model2.predict(X_val)
        val_acc = mean_absolute_error(y_val, y_pred)
        val_mae.append(val_acc)
        val_acc = mean_squared_error(y_val, y_pred)
        val_mse.append(val_acc)
        val_acc = MAPE(y_val, y_pred)
        val_mape.append(val_acc)
        del X_train,y_train,X_val,y_val
        gc.collect()

    # print the mean and standard deviation of the validation mse
    print("initailizer number",i)
    print(f'Validation mae: {np.mean(val_mae):.5f} ± {np.std(val_mae):.5f}')
    print(f'Validation mse: {np.mean(val_mse):.5f} ± {np.std(val_mse):.5f}')
    print(f'Validation mape: {np.mean(val_mape):.5f} ± {np.std(val_mape):.5f}')

700/700 [==============================] - 3s 4ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 3s 4ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 3s 4ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 3s 4ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 3s 3ms/step
initailizer number 0
Validation mae: 0.01042 ± 0.00097
Validation mse: 0.00031 ± 0.00005
Validation mape: 1.11925 ± 0.10542
700/700 [==============================] - 3s 4ms/step
initailizer number 1
Validation mae: 0.07263 ± 0.00011
Validation mse: 0.00605 ± 0.00002
Validation mape: 7.44423 ± 0.00970
700/700 [==============================] - 3s 3ms/step
initailizer number 2
Validation mae: 0.02669 ± 0.00728
Validation mse: 0.00185 ± 0.00074
Validation mape: 2.92122 ± 0.80724
700/700 [==============================] - 3s 4ms/step
initailizer number 3
Validation mae: 0.03025 ± 0.00020
Validation mse: 0.00220 ± 0.00003
Validation mape: 3.31530 ± 0.02363
700/700 [==============================] - 3s 4ms/step
initailizer number 4
Validation mae: 0.00902 ± 0.00056
Validation mse: 0.00020 ± 0.00003
Validation mape: 0.96311 ± 0.06184


In [68]:
X = XT  # your features
y = YT  # your target variable

# set up the k-fold cross-validation object
k =5   # number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)
val_mae = []
val_mse = []
val_mape=[]
i=0

    # loop over the k folds
for train_index, val_index in kf.split(X):
        # split the data into training and validation sets
        X_train, y_train = X[train_index], y[train_index]
        X_val, y_val = X[val_index], y[val_index]
        model2=create_model2(init[0]) #each time we create a model the weights will be initailize with default value given by keras



        # train your model on the training set
        model2.fit(X_train, y_train)
        model2.save("Cnn_lstm_model"+str(input_step)+"_F"+str(i)+".h5")

        # evaluate the model on the validation set and store the mse
        y_pred = model2.predict(X_val)
        val_acc = mean_absolute_error(y_val, y_pred)
        val_mae.append(val_acc)
        val_acc = mean_squared_error(y_val, y_pred)
        val_mse.append(val_acc)
        val_acc = MAPE(y_val, y_pred)
        val_mape.append(val_acc)
        del X_train,y_train,X_val,y_val
        gc.collect()
        i=i+1

    # print the mean and standard deviation of the validation mse
    
print(f'Validation mae: {np.mean(val_mae):.5f} ± {np.std(val_mae):.5f}')
print(f'Validation mse: {np.mean(val_mse):.5f} ± {np.std(val_mse):.5f}')
print(f'Validation mape: {np.mean(val_mape):.5f} ± {np.std(val_mape):.5f}')

700/700 [==============================] - 3s 4ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 3s 4ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 3s 4ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 3s 4ms/step


C:\Users\sarbaoui01\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


700/700 [==============================] - 3s 3ms/step
Validation mae: 0.00915 ± 0.00080
Validation mse: 0.00021 ± 0.00006
Validation mape: 0.97395 ± 0.08786


In [ ]:
val_mae,val_mape